##differentiate MMD

In [1]:
import os
import sys

In [50]:
# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
# Add the parent directory to sys.path
sys.path.append(f"{parent_dir}\\UsedSourceCodes\\wishbone_folder\\wishbone\\src")
sys.path.append(f"{parent_dir}")

In [15]:
import wishbone

here


In [17]:
import numpy as np
import numpy as np
from itertools import permutations

In [29]:
import wishbone
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import keras.backend as K

import tensorflow as tf
#from keras.optimizers import adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import Input, Model
from tensorflow.keras import regularizers
from tensorflow.keras.losses import KLDivergence

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy.stats import wasserstein_distance

#### define custom loss function

In [54]:
from UsedSourceCodes.loss_functions_for_ml_batch_effect import *

<h1>DEFINE MODELS</h1>

In [57]:
def generator(x, y, size, steps_per_epoch):
    for i in range(steps_per_epoch):
        batch_x= x.sample(n = size, replace=True)
        batch_y= y.sample(n = size, replace=True)
        #yield (np.array(batch_x), np.concatenate((np.array(batch_y), np.array(batch_x)), axis =0))   ###why concat
        yield (np.array(batch_x), np.array(batch_y))

In [59]:
@tf.autograph.experimental.do_not_convert
def model_that_corrects_batch_effect(unchanged_df_to_be_used_for_correction_from_batch,
                                     to_be_changed_batch_df,
                                     depth_of_the_model,
                                     epochs,
                                     steps_per_epoch,
                                     cells_per_batch,
                                     val_unchanged_df_to_be_used_for_correction_from_batch,
                                     val_to_be_changed_batch_df,
                                     first_activation,
                                     internal_activation
                                    ):
    l1_lambda = 0.01
    l2_lambda = 0.01
    
    input_shape = to_be_changed_batch_df.shape[1]
    inp = Input((input_shape,), name="input1")
    x = Dense(input_shape, activation=first_activation, kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(inp)
    #x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    n = input_shape
    for i in range(depth_of_the_model):
        n = n + 5
        x = Dense(n, activation= internal_activation, kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(x)
    output1 = Dense(input_shape, activation='relu', name="output11", kernel_regularizer=regularizers.l1(l1_lambda), kernel_initializer='identity')(x)    
    adamoptimizer = tf.keras.optimizers.Adam(learning_rate= 0.001)
    new_model = Model(inputs=inp, outputs=[output1, output1])
    new_model.compile(
                     loss = [mmd2_1,
                              #wasserstein, 
                              d_loss_1, 
                              distance_matrix,
                              #KLDivergence()
                             ],
                     loss_weights= [0.50,
                                    0.30,
                                    0.20,
                                    #0.15, 
                                    #0.15
                                   ],
                
                      #loss= outer_sinkhorn_loss(n = cells_per_batch,niter = epochs, epsilon = 0.001, p=2), 
                       #loss = get_loss,
                      optimizer= adamoptimizer)
    es = EarlyStopping(monitor= "val_loss", mode='min', verbose=2, patience=10)
    if val_unchanged_df_to_be_used_for_correction_from_batch.empty:
                
        hist = new_model.fit_generator(generator(to_be_changed_batch_df, unchanged_df_to_be_used_for_correction_from_batch, 
                                       cells_per_batch, steps_per_epoch*epochs), 
                             steps_per_epoch = steps_per_epoch, 
                             epochs= epochs, 
                            )
    elif not val_unchanged_df_to_be_used_for_correction_from_batch.empty:
        hist = new_model.fit_generator(generator(to_be_changed_batch_df, unchanged_df_to_be_used_for_correction_from_batch, cells_per_batch, steps_per_epoch*epochs), 
                             validation_data = generator(val_to_be_changed_batch_df, val_unchanged_df_to_be_used_for_correction_from_batch, cells_per_batch, ((steps_per_epoch+5)*(epochs+5))),
                             validation_steps = steps_per_epoch,
                             steps_per_epoch = steps_per_epoch, 
                             epochs= epochs, 
                             
                             callbacks=[es]
                            )
        
    return([new_model, hist])

### model_2

In [62]:
###generator
def generator2(x, size, steps_per_epoch):
    for i in range(steps_per_epoch):
        batch_x= x.sample(n = size, replace=True)
        yield np.array(batch_x)

In [64]:
###model
def model_2():
    l1_lambda = 0.01
    l2_lambda = 0.01
    depth_of_the_model = 0
    input_shape = 22  ## hardcoded
    inp = Input((input_shape,), name="input1")
    x = Dense(input_shape, activation='linear', kernel_regularizer=regularizers.l1(l1_lambda))(inp)
    x = tf.keras.layers.BatchNormalization()(x)
    x = Dropout(rate = 0.2)(x)
    n = input_shape
    for i in range(depth_of_the_model):
        n = n + 5
        x = Dense(n, activation='elu', kernel_regularizer=regularizers.l1(l1_lambda))(x)
    output1 = Dense(input_shape, activation='relu', name="output11", kernel_regularizer=regularizers.l1(l1_lambda))(x)    
    adamoptimizer = tf.keras.optimizers.Adam(learning_rate= 0.01)
    new_model = Model(inputs=inp, outputs=output1 )
    new_model.compile(
        loss = [mmd2, d_loss,],
        loss_weights= [0.50, 0.50,],
        #loss = mmd2, 
        optimizer= adamoptimizer)
    return new_model

### Data

In [101]:
parent_parent_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
print(parent_parent_dir)
sys.path.append(f"{parent_parent_dir}")

C:\Users\rajim\Desktop\PHDcodes


In [107]:
path = f"{parent_parent_dir}/data/experiment_UMvsM"
list_files = os.listdir(path)

n1 = 6
new1 = list_files[n1]
print(new1)
new1 = path+"/" + new1
scdata1 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new1),
            cofactor= None)

n2= 8
new2 = list_files[n2]
print(new2)
new2 = path+"/" + new2
scdata2 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new2),
            cofactor= None)

new3 = list_files[n1+1]
print(new3)
new3 = path+"/" + new3
scdata3 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new3),
            cofactor= None)

new4 = list_files[n2+1]
print(new4)
new4 = path+"/" + new4
scdata4 = wishbone.wb.SCData.from_fcs(os.path.expanduser(new4),
            cofactor= None)

1st ag3650_M.fcs
1st ai3661_M.fcs


C:\Users\rajim\AppData\Local\anaconda3\Lib\site-packages\fcsparser\api.py:326: UserWarning: The first two characters were:
 \$. The last two characters were: \ 
Parser expects the same delimiter character in beginning and end of TEXT segment. This file may be parsed incorrectly!
  warnings.warn(msg)
C:\Users\rajim\AppData\Local\anaconda3\Lib\site-packages\fcsparser\api.py:326: UserWarning: The first two characters were:
 \$. The last two characters were: \ 
Parser expects the same delimiter character in beginning and end of TEXT segment. This file may be parsed incorrectly!
  warnings.warn(msg)


1st ah3656_UM.fcs
1st aj3663_M.fcs


C:\Users\rajim\AppData\Local\anaconda3\Lib\site-packages\fcsparser\api.py:326: UserWarning: The first two characters were:
 \$. The last two characters were: \ 
Parser expects the same delimiter character in beginning and end of TEXT segment. This file may be parsed incorrectly!
  warnings.warn(msg)
C:\Users\rajim\AppData\Local\anaconda3\Lib\site-packages\fcsparser\api.py:326: UserWarning: The first two characters were:
 \$. The last two characters were: \ 
Parser expects the same delimiter character in beginning and end of TEXT segment. This file may be parsed incorrectly!
  warnings.warn(msg)


In [129]:
columnstouse = ['141Pr_CD196', '142Nd_CD19', '143Nd_CD5', '144Nd_CD38', '145Nd',
       '146Nd_IgD', '147Sm_CD11c', '148Nd_CD16', '149Sm_CCR4', '150Nd_CD43',
       '151Eu_CD69', '152Sm_CD21', '153Eu_CXCR5', '154Sm_CD62L',
       '155Gd_CD45RA', '158Gd_CD27', '159Tb_CD22', '160Gd_CD14', '162Dy',
       '163Dy_CXCR3', '164Dy_CD23', '165Ho_CD45_BC', '166Er_CD24',
       '167Er_CCR7', '168Er_CD8a', '169Tm_CD45_BC', '170Er_CD3', '171Yb_CD20',
       '172Yb_IgM', '173Yb_HLA-DR', '174Yb_CD49d', '175Lu_CXCR4', '176Yb_CD56']

In [131]:
scdata1.data = scdata1.data[columnstouse]
scdata2.data = scdata2.data[columnstouse]
scdata3.data = scdata3.data[columnstouse]
scdata4.data = scdata4.data[columnstouse]

In [133]:
scaler1 = StandardScaler()
scaler1.fit(scdata1.data)

scaler5 = StandardScaler()
scaler5.fit(scdata3.data)

column_names = scdata1.data.columns

In [135]:
scdata1.normalize()
scdata2.normalize()
scdata3.normalize()
scdata4.normalize()

AttributeError: 'SCData' object has no attribute 'normalize'

In [143]:
from openTSNE import TSNE

In [147]:
tsne = TSNE(n_components=2, perplexity=30, metric="euclidean", random_state=42)
X_embedded = tsne.fit(np.array(scdata1.data))

In [139]:
scdata1.run_tsne()

KeyError: 0

In [ ]:
###data
source_dataset = scdata1.data
source_dataset1 = scdata2.data
target_dataset = scdata3.data
target_dataset2 = scdata4.data

In [ ]:
A = scdata1.data
B = scdata2.data
C = scdata3.data
D = scdata4.data

### get the base mmd value

In [ ]:
base_value_for_anchor = MMD(A, B, 1)
base_value_for_anchor.numpy()

In [ ]:
base_value_for_validation = MMD(C, D, 1)
base_value_for_validation.numpy()

### run model_2

In [ ]:
num_epochs = 30
number_of_cell =2000
source_iterator = generator2(source_dataset, number_of_cell, num_epochs)
source_iterator1 = generator2(source_dataset1, number_of_cell, num_epochs)
target_iterator = generator2(target_dataset, number_of_cell, num_epochs)
target_iterator1 = generator2(target_dataset, number_of_cell, num_epochs)
# Iterate over epochs
new_model = model_2()
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}:")
    source_batch = next(source_iterator)
    source_batch1 = next(source_iterator1)
    loss = new_model.train_on_batch(source_batch, source_batch1)  # Update the model with the source dataset
    print(f'Custom Loss: {loss}')
    
    target_batch = next(target_iterator)
    target_batch1 = next(target_iterator1)
    loss = new_model.train_on_batch(target_batch, target_batch1)  # Update the model with the target dataset
    print(f'Custom Loss: {loss}')



In [ ]:
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
number_of_cells_in_each_iteration = 500
first = f"Batch{N1}"    ###blue
second = f"Batch{N2}"   ###red

In [ ]:
A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)
B= scdata2.data.sample(n = number_of_cells_in_each_iteration, replace=True)
B = B.reset_index(drop=True)
AB_merged = A.append(B, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(new_model.predict(B))
res.columns = B.columns

A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)

AB_merged = A.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

validation

In [ ]:
C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = C.reset_index(drop=True)
D= scdata4.data.sample(n = number_of_cells_in_each_iteration, replace=True)
D = D.reset_index(drop=True)
CD_merged = C.append(D, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(new_model.predict(D))
res.columns = D.columns

C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = A.reset_index(drop=True)

CD_merged = C.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

###
###
###
###
###

### run model_1

In [ ]:
cells_to_use = 1500
model = model_that_corrects_batch_effect(A, B, 1, 10,5, cells_to_use, C, D,
                                         "linear",
                                         "linear"
                                          )

In [ ]:
hist= model[1]
model = model[0]

In [ ]:
model.summary()

In [ ]:
hist.history["loss"]

In [ ]:
#first_one = hist.history["loss"]
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
plt.plot(hist.history["loss"], c = "b", label = "training")
plt.plot([x for x in hist.history["val_loss"]], c = "r", label ="validation")
#plt.plot(first_one, c = "r", label ="validation")

plt.legend()
plt.title(f"Training and validation Losses \nof model for correcting batch effect \n between batches {N1} and {N2}")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.show()


<h1><b>Plot pca before correction of two batches</b></h1>

In [ ]:
N1 = int((n1/2)+1)
N2 = int((n2/2)+1)

In [ ]:
number_of_cells_in_each_iteration = 500
first = f"Batch{N1}"    ###blue
second = f"Batch{N2}"   ###red

In [ ]:
A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)
B= scdata2.data.sample(n = number_of_cells_in_each_iteration, replace=True)
B = B.reset_index(drop=True)
AB_merged = A.append(B, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(model.predict(B)[0])
res.columns = B.columns

A= scdata1.data.sample(n = number_of_cells_in_each_iteration, replace=True)
A = A.reset_index(drop=True)

AB_merged = A.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(AB_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction anchor \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()

validation

In [ ]:
C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = C.reset_index(drop=True)
D= scdata4.data.sample(n = number_of_cells_in_each_iteration, replace=True)
D = D.reset_index(drop=True)
CD_merged = C.append(D, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} before correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()

In [ ]:
res = pd.DataFrame(model.predict(D)[0])
res.columns = D.columns

C= scdata3.data.sample(n = number_of_cells_in_each_iteration, replace=True)
C = A.reset_index(drop=True)

CD_merged = C.append(res, ignore_index=True)
colours = ["b"]*number_of_cells_in_each_iteration + ["r"]*number_of_cells_in_each_iteration
pca = PCA()
Xt = pca.fit_transform(CD_merged)
#plot = plt.scatter(Xt[:,0], Xt[:,1], c = colours)
plt.scatter(Xt[:number_of_cells_in_each_iteration,0], Xt[:number_of_cells_in_each_iteration,1], c = colours[:number_of_cells_in_each_iteration], label = first)
plt.scatter(Xt[number_of_cells_in_each_iteration+1:,0], Xt[number_of_cells_in_each_iteration+1:,1], c = colours[number_of_cells_in_each_iteration+1:], label = second)
plt.title(f"{first} vs {second} after correction validation \nsamples")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()

plt.show()